# The University of Hong Kong
## DASC7600 Data Science Project 2024
## Discrete Compartmental Model

# Import Modules and Settings

In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from scipy.signal import find_peaks, peak_widths

import covid_module
from discrete_compartmental_model import *

# Settings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
# Read csv file
covid_hk_case_cnt_std = pd.read_csv('./data/std_data/hk/covid_hk_case_count_std.csv')

# Datatime Data Type and Index

In [3]:
# Modify data type for datatime column
covid_hk_case_cnt_std['report_date'] = pd.to_datetime(covid_hk_case_cnt_std['report_date'], format='%Y%m%d')

# Set Index
covid_hk_case_cnt_std = covid_hk_case_cnt_std.set_index('report_date', drop=False)

# New Case Counts

In [4]:
covid_hk_case_cnt_std['new_case_cnt'] = covid_hk_case_cnt_std['cuml_case_cnt'].diff().fillna(0)
covid_hk_case_cnt_std['new_recover_cnt'] = covid_hk_case_cnt_std['cuml_dischg_cnt'].diff().fillna(0)

Due to the differences in scale between the first four waves and the fifth wave of Covid-19, <br>
It is not appropriate to plot the counts on the same chart. Therefore, we will plot their graphs separately.

# Waves Automated Detection

In [5]:
# According to the Wikipedia (https://zh.wikipedia.org/wiki/2019%E5%86%A0%E7%8B%80%E7%97%85%E6%AF%92%E7%97%85%E9%A6%99%E6%B8%AF%E7%96%AB%E6%83%85),
# The periods of the Cov-19 waves in Hong Kong are listed as follows:
# 1st Wave: Late January to late February 2020（2020年1月下旬至2月下旬）
# 2nd Wave: Mid-March to Late April 2020（2020年3月中旬至4月下旬）
# 3rd Wave: Early July to late September 2020（2020年7月上旬至9月下旬）
# 4th Wave: Late November 2020 to late May 2021（2020年11月下旬至2021年5月下旬）
# 5th Wave: Late December 2021 to March 2023（2021年12月下旬至2023年3月）

In [6]:
# Counts of first 4 waves
first_4_wave_cnt = covid_hk_case_cnt_std[covid_hk_case_cnt_std['report_date'] <= datetime.datetime(2021,12,20)][['new_case_cnt', 'new_recover_cnt']]

# Peaks and widths
## Return: peaks (ndarray), properties (dict)
first_4_wave_peaks_index, _ = find_peaks(first_4_wave_cnt['new_case_cnt'], width=5)
## Return: widths (ndarray), width_heights (ndarray), left_ips (ndarray), right_ips (ndarray)
first_4_wave_widths = np.array(peak_widths(first_4_wave_cnt['new_case_cnt'], first_4_wave_peaks_index, rel_height=1), 'int') 
## Start and end datetimes
first_4_wave_start_dt_index = first_4_wave_cnt.index[first_4_wave_widths[2]] + datetime.timedelta(days=1)
first_4_wave_end_dt_index = first_4_wave_cnt.index[first_4_wave_widths[3]]

In [7]:
# # Plot the new case counts with different waves identified
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(first_4_wave_cnt.index, first_4_wave_cnt['new_case_cnt'])
# ## x-axis
# plt.plot(first_4_wave_cnt.index, np.zeros(first_4_wave_cnt.shape[0]), '--', color='gray')
# ## Peak of each wave
# plt.plot(first_4_wave_cnt.index[first_4_wave_peaks_index], first_4_wave_cnt['new_case_cnt'][first_4_wave_peaks_index], 'x', color='g')
# ## Period of each wave
# plt.plot(first_4_wave_start_dt_index, first_4_wave_widths[1], '|', color='g')
# plt.plot(first_4_wave_end_dt_index, first_4_wave_widths[1], '|', color='g')
# plt.hlines(first_4_wave_widths[1], first_4_wave_start_dt_index, first_4_wave_end_dt_index, color='g')
# ## Title, x-axis label, y-axis label
# plt.title('Covid-19 waves identified (Hong Kong)')
# plt.xlabel('Date')
# plt.ylabel('Number of New Cases')
# plt.show()

In [8]:
print(f'The highest new case count until late Februrary 2020 is \
{int(covid_hk_case_cnt_std[covid_hk_case_cnt_std["report_date"] < datetime.datetime(2020,3,1)][["new_case_cnt"]].max())}.')

The highest new case count until late Februrary 2020 is 10.


Only the 2nd, 3rd, and 4th waves are identified by the program. <br>
This is likely because the 1st wave does not have a distinct peak, with the highest count being only 10 until late February 2020.

In [9]:
# Counts of the fifth wave
fifth_wave_cnt = covid_hk_case_cnt_std[covid_hk_case_cnt_std['report_date'] > datetime.datetime(2021,12,20)][['new_case_cnt', 'new_recover_cnt']]

# Peaks and Widths
## Return: peaks (ndarray), properties (dict)
fifth_wave_peaks_index, _ = find_peaks(fifth_wave_cnt['new_case_cnt'], width=5, prominence=10000)
## Return: widths (ndarray), width_heights (ndarray), left_ips (ndarray), right_ips (ndarray)
fifth_wave_widths = np.array(peak_widths(fifth_wave_cnt['new_case_cnt'], fifth_wave_peaks_index, rel_height=1), 'int') 
# Start and end datetimes
fifth_wave_start_dt_index = fifth_wave_cnt.index[fifth_wave_widths[2]] + datetime.timedelta(days=1)
fifth_wave_end_dt_index = fifth_wave_cnt.index[fifth_wave_widths[3]]

In [10]:
# # Plot the new case counts with different waves identified
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(fifth_wave_cnt.index, fifth_wave_cnt['new_case_cnt'])
# ## x-axis
# plt.plot(fifth_wave_cnt.index, np.zeros(fifth_wave_cnt.shape[0]), '--', color='gray')
# ## Peak of each wave
# plt.plot(fifth_wave_cnt.index[fifth_wave_peaks_index], fifth_wave_cnt['new_case_cnt'][fifth_wave_peaks_index], 'x', color='g')
# ## Period of each wave
# plt.plot(fifth_wave_start_dt_index, fifth_wave_widths[1], '|', color='g')
# plt.plot(fifth_wave_end_dt_index, fifth_wave_widths[1], '|', color='g')
# plt.hlines(fifth_wave_widths[1], fifth_wave_start_dt_index, fifth_wave_end_dt_index, color='g')
# ## Title, x-axis label, y-axis label
# plt.title('Covid-19 waves identified (Hong Kong)')
# plt.xlabel('Date')
# plt.ylabel('Number of New Cases')
# plt.show()

# Models Fitting - Second Wave

In [11]:
print(f'As mentioned above,\n\
The 2nd wave (from {first_4_wave_start_dt_index[0].strftime("%Y-%m-%d")} to {first_4_wave_end_dt_index[0].strftime("%Y-%m-%d")}) of Covid-19 \
is the first wave identified by the program.\n\
We will now begin fitting different compartmental models (SIR, SIS, SIRS, and SEIR) to the data from the 2nd wave.')

As mentioned above,
The 2nd wave (from 2020-03-01 to 2020-04-20) of Covid-19 is the first wave identified by the program.
We will now begin fitting different compartmental models (SIR, SIS, SIRS, and SEIR) to the data from the 2nd wave.


In [12]:
# New case counts for 2nd wave
second_wave_case_cnt_df = first_4_wave_cnt[first_4_wave_start_dt_index[0]:first_4_wave_end_dt_index[0]]
second_wave_index = second_wave_case_cnt_df.index
second_wave_case_cnt = second_wave_case_cnt_df.values.T

In [13]:
# # SIR Model
# Discrete_SIR_model_second_wave = Discrete_SIR_model()

# Discrete_SIR_model_second_wave.fit(second_wave_case_cnt,
#                                    [sum(second_wave_case_cnt[0][1:]),
#                                     second_wave_case_cnt[0][0],
#                                     second_wave_case_cnt[1][0]])

# plot_compartmental_model_result(second_wave_index, Discrete_SIR_model_second_wave, second_wave_case_cnt)

In [14]:
# # SIS Model
# Discrete_SIS_model_second_wave = Discrete_SIS_model()

# Discrete_SIS_model_second_wave \
#     .fit(second_wave_case_cnt,
#         [sum(second_wave_case_cnt[0][1:]),
#          second_wave_case_cnt[0][0]])

# plot_compartmental_model_result(second_wave_index, Discrete_SIS_model_second_wave, second_wave_case_cnt)

In [15]:
# # SIRS Model
# ## Fix with true_I_plus counts only due to a lack of information about stage changing from R to S
# Discrete_SIRS_model_second_wave = Discrete_SIRS_model()

# Discrete_SIRS_model_second_wave \
#     .fit(second_wave_case_cnt[[0]],
#          [sum(second_wave_case_cnt[0][1:]),
#           second_wave_case_cnt[0][0],
#           second_wave_case_cnt[1][0]])

# plot_compartmental_model_result(second_wave_index, Discrete_SIRS_model_second_wave, second_wave_case_cnt)

In [16]:
# # SEIR Model
# ## Fix with true_I_plus counts only due to a lack of information about stage E
# Discrete_SEIR_model_second_wave = Discrete_SEIR_model()

# Discrete_SEIR_model_second_wave \
#     .fit(second_wave_case_cnt[[0]],
#          [sum(second_wave_case_cnt[0][1:]) - second_wave_case_cnt[0][0],
#           second_wave_case_cnt[0][0],
#           second_wave_case_cnt[0][0],
#           second_wave_case_cnt[1][0]])

# plot_compartmental_model_result(second_wave_index, Discrete_SEIR_model_second_wave, second_wave_case_cnt)

# SEIR Model Fitting

In [17]:
nbr_of_wave = len(first_4_wave_peaks_index)

first_4_wave_index_list = []
first_4_wave_predicted_cnt_list = []
for i in range(nbr_of_wave):
    wave_new_case_cnt_df = first_4_wave_cnt[first_4_wave_start_dt_index[i]:first_4_wave_end_dt_index[i]]
    wave_index = wave_new_case_cnt_df.index
    wave_new_case_cnt = wave_new_case_cnt_df.values.T
    
    wave_SEIR_model = Discrete_SEIR_model()
    
    wave_SEIR_model \
        .fit(wave_new_case_cnt[[0]],
             [sum(wave_new_case_cnt[0][1:]) - wave_new_case_cnt[0][0],
              wave_new_case_cnt[0][0],
              wave_new_case_cnt[0][0],
              wave_new_case_cnt[1][0]])
    
    first_4_wave_index_list.append(wave_index)
    first_4_wave_predicted_cnt_list.append(wave_SEIR_model.output[0][1])

Fitting compartmental model ...
The optimal parameters found are: [0.45993674 0.48143814 0.13074271]
Generating the counts as model output ...
Completed.
Fitting compartmental model ...
The optimal parameters found are: [0.33275816 0.56306525 0.13650382]
Generating the counts as model output ...
Completed.
Fitting compartmental model ...
The optimal parameters found are: [0.24642568 0.5603705  0.14251331]
Generating the counts as model output ...
Completed.


In [18]:
# # Plot the new case counts with different waves identified and their predicted counts
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(first_4_wave_cnt.index, first_4_wave_cnt['new_case_cnt'])
# ## x-axis
# plt.plot(first_4_wave_cnt.index, np.zeros(first_4_wave_cnt.shape[0]), '--', color='gray')
# ## Peak of each wave
# plt.plot(first_4_wave_cnt.index[first_4_wave_peaks_index], first_4_wave_cnt['new_case_cnt'][first_4_wave_peaks_index], 'x', color='g')
# ## Period of each wave
# plt.plot(first_4_wave_start_dt_index, first_4_wave_widths[1], '|', color='g')
# plt.plot(first_4_wave_end_dt_index, first_4_wave_widths[1], '|', color='g')
# plt.hlines(first_4_wave_widths[1], first_4_wave_start_dt_index, first_4_wave_end_dt_index, color='g')
# ## Title
# plt.title('Covid-19 waves identified (Hong Kong)')
# plt.xlabel('Date')
# plt.ylabel('Number of New Cases')
# # Predicted new case counts counts
# for i in range(nbr_of_wave):
#     plt.plot(first_4_wave_index_list[i], first_4_wave_predicted_cnt_list[i], color='red')
# plt.show()

In [19]:
nbr_of_wave = len(fifth_wave_peaks_index)

fifth_wave_index_list = []
fifth_wave_predicted_cnt_list = []
for i in range(nbr_of_wave):
    wave_new_case_cnt = fifth_wave_cnt[fifth_wave_start_dt_index[i]:fifth_wave_end_dt_index[i]]
    wave_index = wave_new_case_cnt.index
    wave_new_case_cnt = wave_new_case_cnt.values.T
    
    wave_SEIR_model = Discrete_SEIR_model()
    
    wave_SEIR_model.fit(wave_new_case_cnt[[0]],
                        [sum(wave_new_case_cnt[0][1:]) - wave_new_case_cnt[0][0],
                         wave_new_case_cnt[0][0],
                         wave_new_case_cnt[0][0],
                         wave_new_case_cnt[1][0]])
    
    fifth_wave_index_list.append(wave_index)
    fifth_wave_predicted_cnt_list.append(wave_SEIR_model.output[0][1])

Fitting compartmental model ...
The optimal parameters found are: [0.67870807 0.51622086 0.11246046]
Generating the counts as model output ...
Completed.


In [20]:
# # Plot the new case counts with different waves identified and their predicted counts
# plt.subplots(figsize=(15, 6))
# ## New Case counts
# plt.plot(fifth_wave_cnt.index, fifth_wave_cnt['new_case_cnt'])
# ## x-axis
# plt.plot(fifth_wave_cnt.index, np.zeros(fifth_wave_cnt.shape[0]), '--', color='gray')
# ## Peak of each wave
# plt.plot(fifth_wave_cnt.index[fifth_wave_peaks_index], fifth_wave_cnt['new_case_cnt'][fifth_wave_peaks_index], 'x', color='g')
# ## Period of each wave
# plt.plot(fifth_wave_start_dt_index, fifth_wave_widths[1], '|', color='g')
# plt.plot(fifth_wave_end_dt_index, fifth_wave_widths[1], '|', color='g')
# plt.hlines(fifth_wave_widths[1], fifth_wave_start_dt_index, fifth_wave_end_dt_index, color='g')
# ## Title
# plt.title('Covid-19 waves identified (Hong Kong)')
# plt.xlabel('Date')
# plt.ylabel('Number of New Cases')
# # Predicted new case counts counts
# for i in range(nbr_of_wave):
#     plt.plot(fifth_wave_index_list[i], fifth_wave_predicted_cnt_list[i], color='red')
# plt.show()

# 